In [ ]:
import pandas as pd
import altair as alt

In [ ]:
# df = pd.read_csv("tidy_timeliness.csv")

In [ ]:
# Objective:  Line chart of disposal receipt, below diff, below outstanding
url = 'https://gist.githubusercontent.com/RobinL/5a3fb37c43c017a051ea7ece1804d18b/raw/bb0816955fbd30514d993ab7c2cb379837abc41a/tidy_timeliness.csv'

In [ ]:
data = alt.Data(url = url, format = alt.DataFormat(type='csv', parse={'receipts': 'number', 'disposals': 'number', 'outstanding': 'number'}))
data.to_dict(validate=False)
# alt.Chart(df2).mark_bar().encode(x='yearquarter:O', y='sum(receipts):Q')

In [ ]:
r_col = alt.ColorValue('#b74f4f')
d_col = alt.ColorValue('#6b8dda')
o_col = alt.ColorValue('green')

y_axis_r_d = alt.Axis(title='Number of cases per quarter')
y_axis_o = alt.Axis(title='Number of cases at end of quarter')

r_d_scale = alt.Scale(domain=['Disposals', 'Receipts', "Outstanding"], range=['#6b8dda', '#b74f4f', 'green'])

In [ ]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['yearquarter'], empty='none')

In [ ]:
# In the base chart we compute the various fields we want for the chart

rgtd = 'datum.yearquarter + ": Receipts exceeded disposals by "'
dgtr = 'datum.yearquarter + ": Disposals exceeded receipts by "'
rmd = '(datum.receipts_q - datum.disposals_q)'
dmr = '(datum.disposals_q - datum.receipts_q)'

base_chart = alt.Chart(data)\
          .properties(width = 600, 
                      height = 200,
                      title = 'Cases outstanding'
          ).transform_aggregate(
             receipts_q = 'sum(receipts):Q',
             disposals_q = 'sum(disposals):Q',
             outstanding_q = 'sum(outstanding):Q',
             groupby = ['yearquarter']
          ).transform_calculate(
             receipts_less_disposals_q = rmd,
             hover_text = f'{rmd} > 0 ? {rgtd} + {rmd} : {dgtr} + {dmr}',
             legend = f'{rmd} > 0 ? "Receipts" : "Disposals"',
             text_position = f'{rmd} > 0 ? datum.receipts_q : datum.disposals_q',
          )


In [ ]:
chart_o = base_chart\
        .mark_line(
       ).encode(
            x = alt.X("yearquarter:O", axis=alt.Axis(title="", labels=True )),
            y = alt.Y("outstanding_q:Q", axis= y_axis_o),
            color= o_col
)

chart_o_text = base_chart\
        .mark_text(dx = 0, dy = -7
       ).encode(
            x = "yearquarter:O",
            y = "outstanding_q:Q",
            text= "outstanding_q:O",
           opacity = alt.condition(nearest, alt.value(1), alt.value(0)),
    color = o_col
).add_selection(
    nearest
)

chart_top = chart_o + chart_o_text
chart_top

In [ ]:
chart_r = base_chart\
        .properties(title='Receipts and disposals'
       ).mark_line(
       ).encode(
            x = alt.X("yearquarter:O", axis=alt.Axis(title="")),
            y = alt.Y("receipts_q:Q", axis = y_axis_r_d),
            color = r_col
          )
chart_d = chart_r\
        .mark_line(
       ).encode(
            y = "disposals_q:Q",
            color = d_col
          )

chart_rules = base_chart.mark_rule().encode(
    y = "receipts_q:Q",
    y2 = "disposals_q:Q",
    x = "yearquarter:O",
    opacity= alt.condition(nearest, alt.value(1), alt.value(0))
).add_selection(
    nearest
)

chart_t = chart_r.mark_text(dy=-10).encode(
 text = 'hover_text:O',
    y = 'text_position:Q',
 color = alt.Color('legend:N', scale = r_d_scale),
     opacity= alt.condition(nearest, alt.value(1), alt.value(0))
)

chart_bottom = chart_r + chart_d + chart_t + chart_rules


In [ ]:
chart_final = alt.vconcat(chart_top, chart_bottom)
# chart_final.save("o_r_d_chart.vl.json")
chart_final

In [ ]:
df3 = df2.pivot_table(index='yearquarter', values=['receipts', 'disposals', 'outstanding'], aggfunc='sum')
df3['diff'] = df3['receipts'] - df3['disposals']
df3['outstanding_lag'] = df3['outstanding'] - df3['outstanding'].shift(1)
df3

In [ ]:
import altair as alt
import pandas as pd
data = pd.DataFrame({'x': [1, 2, 3], 'y': [2, 1, 2]})

chart = alt.Chart().properties(data=data)
chart = alt.HConcatChart(data)
from pprint import pprint
chart.to_dict(validate=False)


In [ ]:
import altair as alt
from vega_datasets import data

iris = data.iris()

xscale = alt.Scale(domain=(4.0, 8.0))
yscale = alt.Scale(domain=(1.9, 4.55))

area_args = {'opacity': .3, 'interpolate': 'step'}
blank_axis = alt.Axis(title='')

points = alt.Chart(iris).mark_circle().encode(
    alt.X('sepalLength', scale=xscale),
    alt.Y('sepalWidth', scale=yscale),
    color='species',
)

top_hist = alt.Chart(iris).mark_area(**area_args).encode(
    alt.X('sepalLength:Q',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          bin=alt.Bin(maxbins=20, extent=xscale.domain),
          stack=None,
          axis=blank_axis,
         ),
    alt.Y('count()', stack=None, axis=blank_axis),
    alt.Color('species:N'),
).properties(height=60)

right_hist = alt.Chart(iris).mark_area(**area_args).encode(
    alt.Y('sepalWidth:Q',
          bin=alt.Bin(maxbins=20, extent=yscale.domain),
          stack=None,
          axis=blank_axis,
         ),
    alt.X('count()', stack=None, axis=blank_axis),
    alt.Color('species:N'),
).properties(width=60)

c = top_hist & (points | right_hist)
c.to_dict()